In [36]:
import mne
import matplotlib.pyplot as plt
import numpy as np

""" preprocess
    Performs all the preprocessing algorithims on the EEG data
    
    Parameters
    ----------
    eeg : mne raw object
        First Parameter, EEG Data (must include)
    params: dict
        dictionary of all the parameters 
        params = {'line_noise' : 50 \
                'filter_type': None, \
                'filt_freq': None, \
                'filter_length': 'auto', \
                'eog_index': -1, \
                'lam': -1,
                'tol': 1e-7,
                'max_iter': 1000
                }
        values included are default 
    
    Returns
    -------
    Data : double numpy array
        Corrected Data
    fig : matlib figures
        Figures of the data at different processing stages        
    
"""

def preprocess(eeg, params):
    
    #performPrep
    badChannels = ['f','s','g']#prep_pipeline(eeg,params['line_noise'])
    eeg.info['bads'] = badChannels
    
    #perfom filter
    eeg_filt = eeg.copy()
    eeg_filt._data = performFilter(data, eeg.sfreq,\
                             params['filter_type'], \
                             params['filt_freq'],\
                             params['filter_length'])
    
    #eog_regression
    eeg_filt_eog = eeg_filt.copy()
    eeg_filt_eog = performEOGRegression(raw_filt)
    
    #perform RPCA
    eeg_filt_eog_rpca = eeg_filt_eog.copy()
    eeg_filt_eog_rpca._data, noise = performRPCA(raw_filt_eog.get_data(), \
                                    params['lam'], \
                                    params['tol'], \
                                    params['maxIter'])
    
    #Figures 
    
    fig1 = plt.figure(1)
    plt.setp(fig1, color=[1,1,1])
    plt.subplot([11,1,1])
    #EOG Graph
    if 'eeg' in eeg:
        plt.jet()
        eog = raw.pick_types(eog=True)
        scale_min = np.round(np.min(np.min(eog.get_data())))
        scale_max = np.round(np.max(np.max(eog.get_data())))
        plt.clim(scale_min,scale_max)
        plt.title('Filtered EOG data');
        plt.colorbar()
        
    
    

In [29]:
raw = mne.io.read_raw_edf('/Users/raphaelbechtold/Documents/MATLAB/Automagic/automagic/data/Subj1/S001R04.edf')
params = {'line_noise' : 50, \
          'filter_type' : None, \
          'filt_freq' : None, \
          'filter_length' : 'auto', \
          'eog_index' : -1, \
          'lam' : -1,
          'tol' : 1e-7,
          'max_iter': 1000
         }
print(raw.info)

Extracting EDF parameters from /Users/raphaelbechtold/Documents/MATLAB/Automagic/automagic/data/Subj1/S001R04.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
<Info | 16 non-empty fields
    bads : list | 0 items
    ch_names : list | Fc5., Fc3., Fc1., Fcz., Fc2., Fc4., Fc6., C5.., C3.., ...
    chs : list | 64 items (EEG: 64)
    comps : list | 0 items
    custom_ref_applied : bool | False
    dev_head_t : Transform | 3 items
    events : list | 0 items
    highpass : float | 0.0 Hz
    hpi_meas : list | 0 items
    hpi_results : list | 0 items
    lowpass : float | 80.0 Hz
    meas_date : tuple | 2009-08-12 16:15:00 GMT
    nchan : int | 64
    proc_history : list | 0 items
    projs : list | 0 items
    sfreq : float | 160.0 Hz
    acq_pars : NoneType
    acq_stim : NoneType
    ctf_head_t : NoneType
    description : NoneType
    dev_ctf_t : NoneType
    device_info : NoneType
    dig : NoneType
    experimenter : NoneType
    file_id : None

In [41]:
fig1 = plt.figure(1)
#plt.setp(fig1, color='w')
plt.subplot('11,1,1')
#EOG Graph
if 'eeg' in raw:
    plt.jet()
    eog = raw.pick_types(eog=True)
    scale_min = np.round(np.min(np.min(eog.get_data())))
    scale_max = np.round(np.max(np.max(eog.get_data())))
    plt.clim(scale_min,scale_max)
    plt.title('Filtered EOG data');
    plt.colorbar()
        

ValueError: Single argument to subplot must be a 3-digit integer

<Figure size 432x288 with 0 Axes>